In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [2]:
#dataframing the excel file
xls = pd.ExcelFile('Dataset for analysis.xlsx')
df = pd.read_excel(xls, 'Recruitment dataset')

## Data Cleaning

In [3]:
df.shape

(570609, 28)

In [4]:
df.head()

,ID,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,1,527200.0,178760.0,-0.169101,51.493429,1,3,2,1,2012-01-19,...,6,2,5,0,0,5,4,1,1,E01002821
1,2,524930.0,181430.0,-0.200838,51.517931,1,3,2,1,2012-01-04,...,3,4,6,0,0,0,4,1,1,E01004760
2,3,525860.0,178080.0,-0.188636,51.487618,1,3,2,1,2012-01-10,...,6,4,6,0,0,4,1,1,1,E01002893
3,4,524980.0,181030.0,-0.200259,51.514325,1,3,1,1,2012-01-18,...,3,4,6,0,0,0,1,1,1,E01002886
4,5,526170.0,179200.0,-0.183773,51.497614,1,3,1,1,2012-01-17,...,3,4,6,0,0,0,4,1,1,E01002890


In [5]:
df.isnull().sum()

ID                                                 0
Location_Easting_OSGR                             27
Location_Northing_OSGR                            27
Longitude                                         27
Latitude                                          27
Police_Force                                       0
Accident_Severity                                  0
Number_of_Vehicles                                 0
Number_of_Casualties                               0
Date                                               0
Day_of_Week                                        0
Time                                              31
Local_Authority_(District)                         0
Local_Authority_(Highway)                          0
1st_Road_Class                                     0
1st_Road_Number                                    0
Road_Type                                          0
Speed_limit                                        0
Junction_Detail                               

In [6]:
#other than nulls, we have missing/out of bounds data as well. Which are checked against the data dictionary below.

In [7]:
print(len(df[df['Pedestrian_Crossing-Human_Control']<0]))
print(len(df[df['Pedestrian_Crossing-Physical_Facilities']<0]))
print(len(df[df['Light_Conditions']<0]))
print(len(df[df['Junction_Detail']<0]))
print(len(df[df['Junction_Control']<0]))
print(len(df[df['Urban_or_Rural_Area']>2]))
print(len(df[df['Road_Type']<0]))
print(len(df[df['Did_Police_Officer_Attend_Scene_of_Accident'] < 0]))

140
127
0
9
223675
0
0
9


In [8]:
#LSOA of Accident Location and Junction Control have too many nulls and therefore shall not be considered.
# As for the other null, theyre small enough in our extremely large dataset and therefore shall be dropped

In [9]:
#dropping the unnecessary columns
df.drop(columns = ['Junction_Control','LSOA_of_Accident_Location'], inplace= True)

In [10]:
#dropping the object nulls
df.drop(df[df['Junction_Detail'] < 0].index, inplace=True)
df.drop(df[df['Pedestrian_Crossing-Physical_Facilities'] < 0].index, inplace=True)
df.drop(df[df['Pedestrian_Crossing-Human_Control'] < 0].index, inplace=True)
df.drop(df[df['Did_Police_Officer_Attend_Scene_of_Accident'] < 0].index, inplace=True)

In [11]:
#drop our actual nulls
df.dropna(inplace=True)

In [12]:
df.shape

(570383, 26)

In [13]:
df.dtypes

ID                                                      int64
Location_Easting_OSGR                                 float64
Location_Northing_OSGR                                float64
Longitude                                             float64
Latitude                                              float64
Police_Force                                            int64
Accident_Severity                                       int64
Number_of_Vehicles                                      int64
Number_of_Casualties                                    int64
Date                                           datetime64[ns]
Day_of_Week                                             int64
Time                                                   object
Local_Authority_(District)                              int64
Local_Authority_(Highway)                              object
1st_Road_Class                                          int64
1st_Road_Number                                         int64
Road_Typ

In [14]:
#we can start feature engineering here with the date and time
df['hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour
df['month'] = pd.to_datetime(df['Date'], format='%D:%M:%Y').dt.month

The vast majority of these numbered features are label encoded introduces a meaningless order to it. As such, they should be imputed with their original strings and then subsequently one hot encoded. The values and their corresponding strings are found in the data dictionary.

In [15]:
#imported the data dictionary and split them to separate dataframes for referral
xlsx = pd.ExcelFile('Data_Dictionary.xlsx')
Urban_Rural = pd.read_excel(xlsx, 'Urban_Rural')
Road_Type = pd.read_excel(xlsx, 'Road_Type')
Road_Class = pd.read_excel(xlsx, 'Road_Class')
Police_Officer_Attend = pd.read_excel(xlsx, 'Police_Officer_Attend')
Police_Force = pd.read_excel(xlsx, 'Police_Force')
Ped_Cross_Physical = pd.read_excel(xlsx, 'Ped_Cross_Physical')
Ped_Cross_Human = pd.read_excel(xlsx, 'Ped_Cross_Human')
Local_Authority_Highway = pd.read_excel(xlsx, 'Local_Authority_Highway')
Local_Authority_District = pd.read_excel(xlsx, 'Road_Type')
Local_Authority_District = pd.read_excel(xlsx, 'Local_Authority_District')
Light_Conditions = pd.read_excel(xlsx, 'Light_Conditions')
Junction_Detail = pd.read_excel(xlsx, 'Junction_Detail')
Day_of_Week = pd.read_excel(xlsx, 'Day_of_Week')
Accident_Severity = pd.read_excel(xlsx, 'Accident_Severity')


However after much though and due to the time constraints, it doesnt make sense to undo a label encoded dataset. If it has no ordinal value then it should be one hot encoded as such.

However our predictors such as severity and police attended should be encoded properly.

The districts and regions would help with visualisations though, as such will be replaced with their former nominal values.

In [16]:
#replacing absence with 0
df['Did_Police_Officer_Attend_Scene_of_Accident'] = df['Did_Police_Officer_Attend_Scene_of_Accident'].replace({2:0, 3:0})

In [17]:
#1 as fatal doesnt make sense as the number should increase with severity
df['Accident_Severity'] = df['Accident_Severity'].replace({1:3, 3:1})

In [18]:
df['Did_Police_Officer_Attend_Scene_of_Accident'].unique()

array([1, 0], dtype=int64)

In [19]:
Police_Force.set_index('code', inplace=True)
policesector = Police_Force.to_dict()
replace_dict_1 = policesector['label']
df['Police_Force'] = df['Police_Force'].replace(replace_dict_1)

In [20]:
Local_Authority_District.set_index('code', inplace=True)
district = Local_Authority_District.to_dict()
replace_dict_2 = district['label']
df['Local_Authority_(District)'] = df['Local_Authority_(District)'].replace(replace_dict_2)

In [35]:
df.to_csv('cleaned_data.csv', index = False)

## One Hot Encoding our Nominal Features

In [19]:
df.columns

Index(['ID', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude',
       'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Date', 'Day_of_Week', 'Time',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'hour', 'month'],
      dtype='object')

Let's separate the nominal from the encoded-object type

In [20]:
df[['Police_Force','Day_of_Week',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions','hour', 'month'
       ]]=df[['Police_Force','Day_of_Week',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'hour', 'month'
       ]].astype(str)

In [21]:
df.dtypes

ID                                                      int64
Location_Easting_OSGR                                 float64
Location_Northing_OSGR                                float64
Longitude                                             float64
Latitude                                              float64
Police_Force                                           object
Accident_Severity                                       int64
Number_of_Vehicles                                      int64
Number_of_Casualties                                    int64
Date                                           datetime64[ns]
Day_of_Week                                            object
Time                                                   object
Local_Authority_(District)                             object
Local_Authority_(Highway)                              object
1st_Road_Class                                         object
1st_Road_Number                                        object
Road_Typ

In [22]:
#we can drop what we dont need and doesnt help in our modelling
df.drop(columns = ['Date', 'Time', '2nd_Road_Number', 'ID','Location_Easting_OSGR',
                   'Location_Northing_OSGR', 'Longitude', 'Latitude', '1st_Road_Number',
                   'Local_Authority_(District)','Local_Authority_(Highway)' ], inplace= True)

In [23]:
object_typed = df.select_dtypes(include = 'object').columns.tolist()

In [24]:
int_typed = df.select_dtypes(include = 'int64').columns.tolist()

In [25]:
df_encoded = df[int_typed]

In [26]:
object_typed

['Police_Force',
 'Day_of_Week',
 '1st_Road_Class',
 'Road_Type',
 'Speed_limit',
 'Junction_Detail',
 '2nd_Road_Class',
 'Pedestrian_Crossing-Human_Control',
 'Pedestrian_Crossing-Physical_Facilities',
 'Light_Conditions',
 'hour',
 'month']

In [27]:
nominal = pd.get_dummies(df[object_typed],drop_first=True)

In [28]:
nominal.shape

(570383, 132)

In [29]:
df_encoded.shape

(570383, 5)

In [30]:
df.shape

(570383, 17)

In [31]:
df_encoded.reset_index(inplace=True, drop=True)
nominal.reset_index(inplace=True, drop=True)

In [32]:
df_encoded

,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident
0,1,2,1,1,1
1,1,2,1,1,1
2,1,2,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
570378,1,3,1,2,1
570379,1,2,1,2,1
570380,2,1,4,2,1
570381,1,3,3,2,1


In [33]:
nominal

,Police_Force_10,Police_Force_11,Police_Force_12,Police_Force_13,Police_Force_14,Police_Force_16,Police_Force_17,Police_Force_20,Police_Force_21,Police_Force_22,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570378,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570379,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570380,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570381,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [34]:
data = pd.concat([df_encoded, nominal], axis=1)

In [35]:
data.shape

(570383, 137)

In [36]:
data

,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,Police_Force_10,Police_Force_11,Police_Force_12,Police_Force_13,Police_Force_14,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,1,2,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570378,1,3,1,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570379,1,2,1,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570380,2,1,4,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
570381,1,3,3,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [38]:
data.columns.tolist()

['Accident_Severity',
 'Number_of_Vehicles',
 'Number_of_Casualties',
 'Urban_or_Rural_Area',
 'Did_Police_Officer_Attend_Scene_of_Accident',
 'Police_Force_10',
 'Police_Force_11',
 'Police_Force_12',
 'Police_Force_13',
 'Police_Force_14',
 'Police_Force_16',
 'Police_Force_17',
 'Police_Force_20',
 'Police_Force_21',
 'Police_Force_22',
 'Police_Force_23',
 'Police_Force_3',
 'Police_Force_30',
 'Police_Force_31',
 'Police_Force_32',
 'Police_Force_33',
 'Police_Force_34',
 'Police_Force_35',
 'Police_Force_36',
 'Police_Force_37',
 'Police_Force_4',
 'Police_Force_40',
 'Police_Force_41',
 'Police_Force_42',
 'Police_Force_43',
 'Police_Force_44',
 'Police_Force_45',
 'Police_Force_46',
 'Police_Force_47',
 'Police_Force_48',
 'Police_Force_5',
 'Police_Force_50',
 'Police_Force_52',
 'Police_Force_53',
 'Police_Force_54',
 'Police_Force_55',
 'Police_Force_6',
 'Police_Force_60',
 'Police_Force_61',
 'Police_Force_62',
 'Police_Force_63',
 'Police_Force_7',
 'Police_Force_91',
 'P

In [39]:
data.to_csv('clean_encoded.csv', index = False)